# 1.1.4.1: Sistemas de Ecuaciones Lineales y Regresión

## Objetivos de Aprendizaje

Al completar este notebook, serás capaz de:

- **Formular** un problema de regresión lineal simple y múltiple como un sistema de ecuaciones en la forma matricial $X\vec{\beta} = \vec{y}$.
- **Construir** la **matriz de diseño** $X$ (incluyendo el término de intercepto) y el vector objetivo $\vec{y}$ a partir de un dataset.
- **Resolver** sistemas sobredeterminados usando la solución de **mínimos cuadrados** con `np.linalg.lstsq`.
- **Interpretar** los coeficientes $\vec{\beta}$ resultantes en el contexto de un problema de datos.
- **Comprender** y diagnosticar la **inestabilidad numérica** causada por la multicolinealidad usando el **número de condición**.

In [ ]:
# --- Celda de Configuración (Oculta) ---
%display latex
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme(style="whitegrid")

--- 
## ⚙️ El Arsenal de Datasets: Nuestra Fuente de Ejercicios

Este notebook es la aplicación directa de los sistemas de ecuaciones al Machine Learning. Usaremos datasets para plantear problemas de regresión realistas y para explorar los desafíos que surgen en la práctica, como la multicolinealidad.

In [ ]:
# === CONFIGURACIÓN DE DATASETS ===
from src.data_generation.create_student_performance import create_student_performance_data
from src.data_generation.create_business_data import create_business_data
from src.data_generation.create_edge_cases import create_edge_cases

# Configuración centralizada de aleatoriedad para REPRODUCIBILIDAD
rng = np.random.default_rng(seed=42)

# === Generación de Datasets y Matrices para este Notebook ===

# 💡 CONTEXTO PEDAGÓGICO: Hilo Conductor (Regresión Lineal Simple)
# El problema de predecir la calificación a partir de las horas de estudio es el ejemplo
# canónico para introducir la formulación matricial de la regresión lineal.
datos_estudiantes = create_student_performance_data(rng, simplified=True, n_samples=100)

# 💡 CONTEXTO PEDAGÓGICO: Regresión Múltiple
# Para ir más allá de la línea recta, usaremos un problema de negocio donde predecimos
# las ventas a partir de múltiples variables (precio, marketing), lo que requiere una matriz X más ancha.
datos_negocio = create_business_data(rng, n_samples=150)

# 💡 CONTEXTO PEDAGÓGICO: Inestabilidad Numérica
# Para entender por qué la multicolinealidad es un problema, generaremos un dataset donde
# las features están altamente correlacionadas. Esto nos permitirá ver el efecto en el número de condición.
datos_multicolineales = create_edge_cases(rng, case_type='multicollinear', n_samples=100)

print("Datasets generados y listos para usar.")

## 1. De la Regresión a un Sistema de Ecuaciones

En ciencia de datos, a menudo queremos modelar una variable objetivo $y$ en función de una o más variables predictoras (features) $x$. El modelo más simple es la **regresión lineal**, que busca encontrar la línea (o hiperplano en más dimensiones) que mejor se ajuste a los datos.

La ecuación para una **regresión lineal simple** es:
$$ y_i = \beta_0 + \beta_1 x_i + \epsilon_i $$
Donde $\beta_0$ es la ordenada en el origen (intercepto), $\beta_1$ es la pendiente y $\epsilon_i$ es el error (la distancia vertical del punto a la línea).

Nuestro objetivo es encontrar los mejores coeficientes $\vec{\beta} = [\beta_0, \beta_1]$ que minimicen este error para todos los puntos de datos a la vez.

### Formulando el Sistema $X\vec{\beta} = \vec{y}$

Para cada una de nuestras $m$ observaciones (estudiantes), podemos escribir una ecuación. Si las apilamos, obtenemos un sistema sobredeterminado (más ecuaciones que incógnitas) que podemos escribir en forma matricial:
$$ 
\underbrace{\begin{pmatrix}
1 & x_{1} \\
1 & x_{2} \\
\vdots & \vdots \\
1 & x_{m}
\end{pmatrix}}_{X \text{ (Matriz de Diseño, } m \times 2)}
\underbrace{\begin{pmatrix} \beta_0 \\ \beta_1 \end{pmatrix}}_{\vec{\beta} \text{ (Vector de Coeficientes, } 2 \times 1)}
\approx
\underbrace{\begin{pmatrix}
y_1 \\
y_2 \\
\vdots \\
y_{m}
\end{pmatrix}}_{\vec{y} \text{ (Vector Objetivo, } m \times 1)}
$$
La columna de unos en la **matriz de diseño $X$** es un truco matemático para que el coeficiente $\beta_0$ (el intercepto) se pueda incluir limpiamente en la multiplicación matricial.

### Ejemplo Demostrativo 1: Regresión Lineal Simple con el Hilo Conductor

In [ ]:
# 1. DATOS: Extraemos 'horas_estudio' como nuestra feature y 'calificacion_examen' como nuestro objetivo.
X_feature = datos_estudiantes[['horas_estudio']].values
y = datos_estudiantes['calificacion_examen'].values

# 2. CONSTRUIR LA MATRIZ DE DISEÑO X: Añadimos la columna de unos para el intercepto.
# np.c_ es una forma conveniente de concatenar columnas.
X = np.c_[np.ones(X_feature.shape[0]), X_feature]

print("Primeras 5 filas de la Matriz de Diseño X:")
print(X[:5])
print(f"\nForma de X: {X.shape}")
print(f"Forma de y: {y.shape}")

# 3. RESOLVER EL SISTEMA: Usamos Mínimos Cuadrados para encontrar la mejor solución aproximada.
# np.linalg.lstsq encuentra el beta que minimiza ||X_beta - y||^2
beta, residuals, rank, s = np.linalg.lstsq(X, y, rcond=None)
beta_0, beta_1 = beta

# 4. INTERPRETACIÓN Y VISUALIZACIÓN
print(f"\nCoeficientes encontrados:")
print(f"  β₀ (intercepto): {beta_0:.2f} -> Calificación esperada con 0 horas de estudio.")
print(f"  β₁ (pendiente):   {beta_1:.2f} -> Puntos adicionales por cada hora de estudio.")

plt.figure(figsize=(10, 6))
plt.scatter(X_feature, y, alpha=0.7, label='Datos Originales')
plt.plot(X_feature, X @ beta, color='red', linewidth=3, label='Línea de Regresión de Mínimos Cuadrados')
plt.title('Regresión Lineal: Calificación vs. Horas de Estudio')
plt.xlabel('Horas de Estudio'); plt.ylabel('Calificación del Examen')
plt.legend(); plt.grid(True)
plt.show()

## 2. Extendiendo a Regresión Múltiple

La belleza del enfoque matricial es que se extiende sin esfuerzo a más dimensiones. Si queremos predecir $y$ a partir de $n$ features, la ecuación se convierte en:
$$ y_i = \beta_0 + \beta_1 x_{i1} + \beta_2 x_{i2} + \dots + \beta_n x_{in} $$
La única diferencia es que nuestra matriz de diseño $X$ ahora tiene más columnas, una por cada feature (más la columna de unos del intercepto). El vector $\vec{\beta}$ también crece, pero el método de solución `np.linalg.lstsq` sigue funcionando exactamente igual.

### Ejemplo Demostrativo 2: Regresión Múltiple con Datos de Negocio

In [ ]:
# 1. DATOS: Predecir 'ventas_mensuales' a partir de 'precio' y 'gasto_marketing'.
features = datos_negocio[['precio', 'gasto_marketing']].values
y = datos_negocio['ventas_mensuales'].values

# 2. CONSTRUIR MATRIZ DE DISEÑO X: Ahora tendrá 3 columnas (intercepto, precio, marketing).
X = np.c_[np.ones(features.shape[0]), features]

print("Primeras 5 filas de la Matriz de Diseño X (múltiple):")
print(np.round(X[:5], 2))
print(f"\nForma de X: {X.shape}")

# 3. RESOLVER EL SISTEMA
beta, _, _, _ = np.linalg.lstsq(X, y, rcond=None)
beta_0, beta_1, beta_2 = beta

# 4. INTERPRETACIÓN
print(f"\nCoeficientes encontrados:")
print(f"  β₀ (base): {beta_0:.2f} -> Ventas base sin precio ni marketing.")
print(f"  β₁ (precio): {beta_1:.2f} -> Cambio en ventas por cada unidad de aumento en el precio.")
print(f"  β₂ (marketing): {beta_2:.2f} -> Cambio en ventas por cada unidad de aumento en marketing.")

## 3. Estabilidad Numérica y Número de Condición

Un problema grave surge cuando las columnas de $X$ son muy similares (multicolinealidad). Esto hace que la matriz $(X^T X)$ (que `lstsq` resuelve internamente) esté mal condicionada, casi singular. La solución $\vec{\beta}$ se vuelve muy sensible a pequeños cambios en los datos, es decir, **inestable** y poco fiable.

El **número de condición** (`np.linalg.cond`) mide esta sensibilidad. 
- Un número **pequeño** (cercano a 1) es **bueno** (matriz bien condicionada).
- Un número **muy grande** (e.g., > 1000) es una **señal de alerta** de multicolinealidad.

### Ejemplo Demostrativo 3: El Peligro de la Multicolinealidad

In [ ]:
# 1. DATOS: Usamos el dataset donde x2 y x3 dependen de x1.
X_mal_condicionado = datos_multicolineales[['x1', 'x2', 'x3']].values
X_bien_condicionado = datos_estudiantes[['horas_estudio', 'calificacion_previa']].values # Del notebook anterior

# 2. APLICACIÓN: Calculamos el número de condición.
cond_mal = np.linalg.cond(X_mal_condicionado)
cond_bien = np.linalg.cond(X_bien_condicionado)

# 3. INTERPRETACIÓN
print(f"Número de Condición (Features Correlacionadas): {cond_mal:,.2f}")
print(f"Número de Condición (Features Independientes): {cond_bien:.2f}")
print("\n🔴 ¡Alerta! Un número de condición tan alto indica que la solución de regresión con estos datos no será fiable.")

---
## 4. Ejercicios Guiados con Scaffolding (8+)
Rellena las partes marcadas con `# COMPLETAR` para afianzar tu comprensión.

### === EJERCICIO GUIADO 1: Construir la Matriz de Diseño X ===

In [ ]:
# DATOS: Un array de NumPy con una sola feature.
feature_x = np.array([5, 8, 12, 15])

# TODO 1: Crea un vector de unos con la misma cantidad de filas que feature_x.
intercepto = # COMPLETAR

# TODO 2: Apila la columna de intercepto y la de feature_x para crear la matriz de diseño X.
# PISTA: Usa np.c_[col1, col2]
X = # COMPLETAR

# VERIFICACIÓN
X_esperada = np.array([[1, 5], [1, 8], [1, 12], [1, 15]])
assert X.shape == (4, 2)
assert np.allclose(X, X_esperada)
print("✅ ¡Matriz de diseño construida correctamente!")
print(X)

### === EJERCICIO GUIADO 2: Resolver un Sistema Simple con Mínimos Cuadrados ===

In [ ]:
# DATOS: La matriz X del ejercicio anterior y un vector objetivo y.
X = np.array([[1, 5], [1, 8], [1, 12], [1, 15]])
y = np.array([12, 18, 25, 30])

# TODO: Usa np.linalg.lstsq para encontrar el vector de coeficientes beta.
# PISTA: La función devuelve varias cosas, pero solo necesitas la primera.
beta, _, _, _ = # COMPLETAR

# VERIFICACIÓN
assert beta.shape == (2,)
assert np.isclose(beta[0], 2.44) and np.isclose(beta[1], 1.84)
print("✅ ¡Sistema resuelto correctamente!")
print(f"β₀ (intercepto) = {beta[0]:.2f}")
print(f"β₁ (pendiente) = {beta[1]:.2f}")

### === EJERCICIO GUIADO 3: Regresión Múltiple con el Hilo Conductor ===

In [ ]:
# DATOS: Usaremos la versión completa de datos_estudiantes.
datos_full = create_student_performance_data(rng, n_samples=50)
features = datos_full[['horas_estudio', 'calificacion_previa']].values
y = datos_full['calificacion_examen'].values

# TODO 1: Construye la matriz de diseño X para este problema de regresión múltiple.
# PISTA: No olvides la columna de unos para el intercepto.
X = # COMPLETAR

# TODO 2: Resuelve el sistema para encontrar los tres coeficientes beta.
beta, _, _, _ = # COMPLETAR

# VERIFICACIÓN
assert X.shape == (50, 3)
assert beta.shape == (3,)
print("✅ ¡Regresión múltiple completada!")
print(f"β₀ (Intercepto): {beta[0]:.2f}")
print(f"β₁ (Horas Estudio): {beta[1]:.2f}")
print(f"β₂ (Calif Previa): {beta[2]:.2f}")

### === EJERCICIO GUIADO 4: Calcular el Número de Condición ===

In [ ]:
# DATOS: La matriz X del ejercicio anterior y la matriz de datos multicolineales.
X_estudiantes = np.c_[np.ones(features.shape[0]), features]
X_multi = datos_multicolineales[['x1', 'x2', 'x3']].values

# TODO 1: Calcula el número de condición de la matriz de estudiantes.
# PISTA: Usa np.linalg.cond().
cond_estudiantes = # COMPLETAR

# TODO 2: Calcula el número de condición de la matriz multicolineal.
cond_multi = # COMPLETAR

# VERIFICACIÓN
assert cond_estudiantes < 1000 # Esperamos que esté bien condicionada
assert cond_multi > 10000 # Esperamos que esté mal condicionada
print("✅ ¡Cálculos correctos!")
print(f"Número de condición (Estudiantes): {cond_estudiantes:.2f} (Estable)")
print(f"Número de condición (Multicolineal): {cond_multi:,.2f} (Inestable)")

--- 
# 5. Banco de Ejercicios Prácticos (30+)
Ahora te toca a ti. Resuelve estos ejercicios para consolidar tu conocimiento.

### Parte A: Sistemas Cuadrados y Estabilidad

**A1 (🟢 Fácil):** Resuelve el sistema 3x3: $x+y+z=6, 2y+5z=-4, 2x+5y-z=27$ usando `np.linalg.solve`.

**A2 (🟢 Fácil):** Calcula el número de condición de la matriz del ejercicio A1. ¿Consideras que es una matriz estable para resolver?

**A3 (🟡 Medio):** Genera una matriz singular 3x3. Intenta resolver un sistema $A\vec{x} = \vec{b}$ con ella. ¿Qué ocurre? ¿Por qué?

### Parte B: Regresión Lineal Simple

**B1 (🟢 Fácil):** Tienes 3 puntos: (1,2), (2,3), (3,5). Construye la matriz de diseño $X$ y el vector $\vec{y}$ para un modelo de regresión lineal.

**B2 (🟢 Fácil):** Resuelve el sistema del ejercicio B1 usando `np.linalg.lstsq` para encontrar la línea de mejor ajuste.

**B3 (🟡 Medio):** Toma una muestra aleatoria de 50 estudiantes de `datos_estudiantes`. Repite el proceso de regresión lineal. Compara los coeficientes $\beta_0, \beta_1$ obtenidos con los del ejemplo demostrativo. ¿Son muy diferentes?

**B4 (🟡 Medio):** Usando el modelo que entrenaste en B3, ¿qué calificación predecirías para un estudiante que estudió 15 horas?

**B5 (🔴 Reto):** Ajusta un modelo de regresión para `datos_negocio` que prediga `ventas_mensuales` basándose únicamente en `precio`. Visualiza los datos y la línea de regresión.

### Parte C: Regresión Múltiple y Retos

**C1 (🟡 Medio):** Construye la matriz de diseño y el vector objetivo para predecir `calificacion_examen` a partir de `horas_estudio` y `asistencia_porcentaje` del dataset completo de estudiantes.

**C2 (🟡 Medio):** Resuelve el sistema del ejercicio C1 y reporta los tres coeficientes $\beta$.

**C3 (🔴 Reto):** Regresión Polinómica. Para ajustar una parábola ($y = \beta_0 + \beta_1 x + \beta_2 x^2$), la matriz de diseño necesita una columna para $x^2$. Toma `datos_estudiantes`, construye una matriz $X$ con las columnas `[1, horas, horas^2]` y resuelve para los tres $\beta$.

**C4 (🔴 Reto):** Visualiza el ajuste parabólico del ejercicio C3 sobre el scatter plot de los datos de estudiantes.

**C5 (🔴 Reto Conceptual):** ¿Por qué una regresión polinómica como la del ejercicio C3 sigue considerándose un problema de *álgebra lineal* y se puede resolver con `lstsq`?

---

## ✅ Mini-Quiz de Autoevaluación

*Responde estas preguntas para verificar tu comprensión.*

1. ¿Por qué `np.linalg.solve` no funciona para la mayoría de los problemas de regresión lineal del mundo real?
2. En la matriz de diseño $X$ para $y = \beta_0 + \beta_1 x_1 + \beta_2 x_2$, ¿qué representa la primera columna de unos?
3. Un número de condición muy alto en la matriz de diseño $X$ es un síntoma de... (completa la palabra).
4. Si tienes $m$ observaciones y $n$ features (incluyendo el intercepto), ¿cuál es la forma de la matriz de diseño $X$?

## 🚀 Próximos Pasos

¡Felicidades! Has implementado un algoritmo de Machine Learning desde cero, conectando directamente la teoría de sistemas de ecuaciones con una aplicación práctica y fundamental.

- En el siguiente notebook, **`1.1.4.2_Proyecciones_y_Minimos_Cuadrados_OLS`**, profundizaremos en la **geometría** detrás de la solución de mínimos cuadrados. Descubriremos cómo el concepto de **proyección ortogonal** nos da esa 'mejor solución aproximada' y derivaremos formalmente la famosa ecuación normal $X^T X \vec{\beta} = X^T \vec{y}$.